## Imports / init

In [ ]:
# Those two lines are for dev only : they watch imported libraries for changes
# %load_ext autoreload
# %autoreload 2

import brightway2 as bw
import os

import lca_algebraic as agb

from sympy import init_printing
import bw2io
from dotenv import load_dotenv

# Pretty print for Sympy
init_printing()

# Init brightway2 and databases

In [ ]:
# Set the current project
# Can be any name
bw.projects.set_current("MyProject")

# It's better to not leave credential in the code.
# Create a file named .env, that you will not share /commit, and contains the following :
# ECOINVENT_LOGIN=<your_login>
# ECOINVENT_PASSWORD=<your_password>

# This load .env file into os.environ
load_dotenv()

# This downloads ecoinvent and installs biopshere + technosphere + LCIA methods
if len(bw.databases) > 0:
    print("Initial setup already done, skipping")
else:
    # This is now the prefered method to init an Brightway2 with Ecoinvent
    # It is not more tied to a specific version of bw2io
    bw2io.import_ecoinvent_release(
        version="3.9",
        system_model="cutoff",
        username=os.environ["ECOINVENT_LOGIN"],  # Read for .env file
        password=os.environ["ECOINVENT_PASSWORD"],  # Read from .env file
        use_mp=True,
    )

In [ ]:
# We use a separate DB for defining our foreground model / activities
# Choose any name
USER_DB = "MyForeground"

# This is better to cleanup the whole foreground model each time, and redefine it in the notebook (or a python file)
# instead of relying on a state or previous run.
# Any persistent state is prone to errors.
agb.resetDb(USER_DB)

# Parameters are stored at project level :
# Reset them also
# You may remove this line if you import a project and parameters from an external source (see loadParam(..))
agb.resetParams()

# Overview of the databases
agb.list_databases()

# Introduction to Numpy

Numpy is a python libray for symbolic calculus. 

You write Sympy expression as you write **standard python expressions**, using **sympy symbols** in them. 

The result is then a **symbolic expression that can be manipulated**, instead of a **numeric value**.

In [ ]:
from sympy import symbols

# create sympy symbol
x = symbols("x")

# Expressions are not directly evaluated
f = x * 2 + 4
f

In [ ]:
# symbols can be replaced by values afterwards
f.subs(dict(x=3))

In practice, you don't need to care about Sympy. Just remember that : 
* The parameters defined below are **instances of sympy symbols**
* Any **valid python expression** containing a **sympy symbol** will create a **sympy symbolic expression**

# Define input parameters

First, we define the input parameters of the model together with their distribution.

The numeric parameters are **instances of sympy 'Symbol'**. 

Thus, any python arithmetic expression composed of parameters will result in a **symbolic expression** to be used later in the definition of the model, rather than a static numeric result.

In [ ]:
# Example of 'float' parameters
a = agb.newFloatParam(
    "a",
    default=0.5,
    min=0.2,
    max=2,
    distrib=agb.DistributionType.TRIANGLE,  # Distribution type, linear by default
    description="hello world",
    label="extended label for a",
)

b = agb.newFloatParam(
    "b",
    default=0.5,  # Fixed if no min /max provided
    distrib=agb.DistributionType.FIXED,
    description="foo bar",
)

share_recycled_aluminium = agb.newFloatParam(
    "share_recycled_aluminium",
    default=0.6,
    min=0,
    max=1,
    std=0.2,
    distrib=agb.DistributionType.NORMAL,  # Normal distrib, with std dev
    description="Share of reycled aluminium",
)

c = agb.newFloatParam("c", default=0.6, std=0.2, distrib=agb.DistributionType.LOGNORMAL)

beta = agb.newFloatParam("beta", default=0.6, std=0.2, a=2, b=5, distrib=agb.DistributionType.BETA)

# You can define boolean parameters, taking only discrete values 0 or 1
bool_param = agb.newBoolParam("bool_param", default=1)

# Example 'enum' parameter, acting like a switch between several possibilities
# Enum parameters are not Symbol themselves
# They are a facility to represent many boolean parameters at once '<paramName>_<enumValue>'
# and should be used with the 'newSwitchAct' method
elec_switch_param = agb.newEnumParam(
    "elec_switch_param",
    values=["us", "eu"],  # If provided as list, all possibilities have te same probability
    default="us",
    description="Switch on electricty mix",
)

# Another example enum param
techno_param = agb.newEnumParam(
    "techno_param",
    values={
        "technoA": 0.4,
        "technoB": 0.1,
        "technoC": 0.5,
    },  # You can provide a statistical weight for each value, by using a dict
    default="technoA",
    description="Choice of technology",
)

## Persistance of parameters

By default, new parameters are kept in memory but also persisted in the project (unless save=False).

You can persist parameters afterwards with `persistParams()`.

You can load also load parameters from an existing database with `loadParams()`.

The persistance of parameters and the distribution is compatible with **Brightway2** and **Activity Browser**  [see documentation of stat_arrays](https://stats-arrays.readthedocs.io/en/latest/)

In [ ]:
# Load parameters previously  persisted in the dabatase.
agb.loadParams()

# Manage several databases

lca_algebraic supports several foreground / background datasets. Background datasets are considered static / non parametrized by the library : they use standard LCA method of **Brightway2**. 

Foreground databases are considered parametric and their activities are developped as functions of parameters and background activities.

## Set status of a database

The functions **setForeground(...)** and **setBackground(...)** change the status of a database.

In [ ]:
agb.setForeground(USER_DB)
agb.list_databases()

## Import / export

`lca_algebraic` extends [BW2Package](https://2.docs.brightway.dev/technical/bw2io.html), adding persistence of parameters.

In [ ]:
_ = agb.findBioAct("*lead")
# agb.findBioAct(name="*water*")